# Should We Buy Singles, or Whole Albums?

Our task in this scenario: the Chinook digital music store has signed a deal with a new record label. We've been trusted by Chinook to select the first 3 albums from the record label. We need to discover which genres sell the most tracks, write a summary of our findings, and recommend which 3 artists we'll buy albums from for the store.

We have 4 artists to choose from: Regal (Hip-Hop), Red Tone (Punk), Meteor and the Girls (Pop), Slim Jim Bites (Blues).

We have other questions to answer as well:

- How are our sales agents performing?
- To which countries should we focus our marketing?
- Should our company buy full albums from record labels, or just single tracks?

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:

%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Now that we've surveyed the tables, we're going to filter for the most popular genres in the USA. Below is code and a table for this purpose.

For reference: the total number of songs sold in the USA from Chinook is 1051. We will use this number in our code below.

In [3]:
%%sql

WITH usa AS
    (
        SELECT *
        FROM invoice
        WHERE billing_country = "USA"
    ),
    songs_bought_usa AS
    (
        SELECT SUM(quantity) songs_bought_usa, il.track_id
        FROM invoice_line il
        INNER JOIN usa ON usa.invoice_id = il.invoice_id
        GROUP BY 2
    ),
    track_usa AS
    (
        SELECT t.genre_id, t.track_id, sbu.songs_bought_usa
        FROM track t
        INNER JOIN songs_bought_usa sbu ON sbu.track_id = t.track_id 
    )
    
SELECT g.name as genre, SUM(tu.songs_bought_usa) songs_bought_usa, ROUND((SUM(tu.songs_bought_usa) / CAST(1051 AS FLOAT)) * 100, 2) AS market_percentage
FROM genre g
INNER JOIN track_usa tu ON tu.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC;


Done.


genre,songs_bought_usa,market_percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


For our relevant artists' genres, Punk (12.37%), Blues (3.43%), and Pop (2.09%) are the largest genres. Hip-hop is just below Pop at 1.9%. 

Punk should definitely be included in the first line-up as it has a large market share by a single order of magnitude. Blues, Pop, and Hip-Hip are very close in terms of market share. At this point, we should think of Chinook's goals, and the record label's.

Does Chinook want to develop a niche for specifc genres, or does it want to involve everything on its platform? Amazon, iTunes, Spotify-- it's difficult to beat the breadth of these digital distributors. However, if Chinook wants to focus on a niche and try to grow that way, serving an underserved area in the market, it would help us choose. If we play the game "Which of these is not like the other?", Hip-Hop has less in common with the other genres. It also has a smaller market share.

It seems best to leave Hip-Hop out with this approach.

However, if we're looking for the strength of charisma in our line-up of artists, the market share of Pop, Blues, and Hip-Hop are close enough to further judge by artist quality. Is Regal the next Kanye West? Probably not, but it's worth a listen and thought.

# How Are Our Sales Agents?

We'll find a sales support agent next. We're going to look for the total dollar amount of sales per sales agent.

Note: total agent sales are 4709 USD

In [4]:
%%sql

WITH agent AS

    (
        SELECT first_name || " " || last_name AS agent_name, employee_id, hire_date
        FROM employee
        WHERE title = "Sales Support Agent"
    ),
    agent_customer AS
    (
        SELECT a.*, customer_id
        FROM customer c
        INNER JOIN agent a ON a.employee_id = c.support_rep_id
    ),
    agent_sales AS
    (
        SELECT ac.agent_name, SUM(i.total) || " USD" AS total_sales, ac.hire_date
        FROM agent_customer ac
        INNER JOIN invoice i ON i.customer_id = ac.customer_id
        GROUP BY ac.agent_name
    )
    
SELECT ags.*, ROUND(total_sales / 4709, 3) AS percentage_of_total_sales
FROM agent_sales ags
GROUP BY ags.agent_name;

Done.


agent_name,total_sales,hire_date,percentage_of_total_sales
Jane Peacock,1731.51 USD,2017-04-01 00:00:00,0.368
Margaret Park,1584.0 USD,2017-05-03 00:00:00,0.336
Steve Johnson,1393.92 USD,2017-10-17 00:00:00,0.296


Each sales agent is around 1/3rd of the total sales with Jane Peacock being the leader. This is likely due to her being hired earlier, having more time to develop her salesmanship.

Overall, our sales agents are performing very closely with each other.

# Analyzing Sales Data By Country

In [5]:
%%sql

WITH country_or_other AS
    (
        SELECT 
            CASE
                WHEN (
                    SELECT COUNT(*)
                    FROM customer
                    WHERE country = c.country
                    ) = 1 THEN "Other"
                ELSE c.country
                END AS country,
                c.customer_id,
                il.*
        FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT country,
        COUNT(DISTINCT(customer_id)) AS customers,
        ROUND(SUM(unit_price)) AS total_sales,
        ROUND(SUM(unit_price) / COUNT(DISTINCT(customer_id)),2) AS average_sale_per_customer,
        ROUND(SUM(unit_price) / COUNT(DISTINCT(invoice_id)),2) AS average_order
FROM country_or_other
GROUP BY 1
ORDER BY 3 DESC;

Done.


country,customers,total_sales,average_sale_per_customer,average_order
Other,15,1095.0,73.0,7.45
USA,13,1040.0,80.04,7.94
Canada,8,536.0,66.95,7.05
Brazil,5,428.0,85.54,7.01
France,5,389.0,77.81,7.78
Germany,4,335.0,83.66,8.16
Czech Republic,2,273.0,136.62,9.11
United Kingdom,3,246.0,81.84,8.77
Portugal,2,185.0,92.57,6.38
India,2,183.0,91.58,8.72


Importante Note: The "Other" category has 15 customers, but each customer in this category belongs to a unique & different country. In terms of influencing marketing campaigns, this category should possess no investment since it is indistinct enough.

In fact, we should say this about the whole data set. It is very, very small. If we were to recommend a course of action based on its data, we would recommend using the average_sale_per_customer column.

- Czech Republic is especially high in average sales per customer, as are Portugal and India. Czech Republic should have first priority in marketing campaigns.

# Should Chinook buy full albums from record labels, or just single tracks?



In [6]:
%%sql

WITH invoice_albums AS
    (
        SELECT
        i.invoice_id,
        COUNT(t.track_id) purchased_track_count,
        COUNT(DISTINCT t.album_id) album_count,
        t.album_id album_id
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        GROUP BY 1
    ),
    album_track_count AS
    (
        SELECT
        t.album_id album_id,
        COUNT(t.track_id) album_track_count
        FROM track t
        GROUP BY 1
    )
    

SELECT  
    CASE
        WHEN ia.album_count = 1
        AND ia.purchased_track_count = atc.album_track_count
        THEN "Albums"
        ELSE "Singles"
        END AS purchase_type,
    COUNT(ia.invoice_id) number_of_invoices,
    ROUND(CAST(COUNT(ia.invoice_id) AS FLOAT) / 
          (
              SELECT COUNT(*)
              FROM invoice), 2) percentage
FROM invoice_albums ia
INNER JOIN album_track_count atc ON atc.album_id = ia.album_id
GROUP BY purchase_type;

Done.


purchase_type,number_of_invoices,percentage
Albums,114,0.19
Singles,500,0.81


Whole albums make 19% of our sales. They should not be changed at all. There is no need to risk losing almost 1/5th of our income.

We should continue to buy full albums from record companies. Customer selection & utility is too important to compromise.